## Etapa 1: Importação de bibliotecas

In [ ]:
!python -m spacy download pt_core_news_sm

In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
from google.colab import drive
import zipfile
import seaborn as sns
import spacy as sp
import string
import random
import matplotlib.pyplot as plt
from nltk import tokenize
import nltk


In [ ]:
nltk.download('rslp') #Utilizado na stemmização

In [ ]:
from wordcloud import WordCloud

In [ ]:
from datetime import datetime

In [ ]:
# ATUALIZAÇÃO AGO-2024
# %tensorflow_version 2.x
import tensorflow as tf
tf.__version__

In [ ]:
from tensorflow.keras import layers
import tensorflow_datasets as tfds

# Dados de Treino:




## Etapa 2: Pré-Processamento de dados: Limpeza

In [ ]:
cols = ['text','label']

In [ ]:
train_data = pd.read_csv('emotions_pt.csv')

In [ ]:
train_data.shape

In [ ]:
train_data.head()

In [ ]:
train_data['label'].unique()

In [ ]:
sns.countplot( x = train_data['label'])

In [ ]:
train_data = train_data.drop(train_data[train_data['label'] == 1].index)

In [ ]:
train_data.replace(2,1, inplace = True)

In [ ]:
sns.countplot( x = train_data['label'])

In [ ]:
data = train_data

In [ ]:
data.head()

In [ ]:
X = data.iloc[:, 0]

In [ ]:
y = data.iloc[:, 1]
print(y)

In [ ]:
print(X)

In [ ]:
data_labels = y
print(y)

Inutil?

In [ ]:
from sklearn.model_selection import train_test_split
X, _, y, _ = train_test_split(X, y, test_size = 0.85, stratify = y)

## Tratativa de linguagem:

In [ ]:
todas_palavras = ' '.join([texto for texto in data['text']])

In [ ]:
token_espaco = tokenize.WhitespaceTokenizer()

token_frase = token_espaco.tokenize(todas_palavras)
frequencia = nltk.FreqDist(token_frase)
frequencia

In [ ]:
def pareto(texto, coluna_texto, quantidade):
    todas_palavras = ' '.join([texto for texto in texto[coluna_texto]])
    token_frase = token_espaco.tokenize(todas_palavras)
    frequencia = nltk.FreqDist(token_frase)
    df_frequencia = pd.DataFrame({"Palavra": list(frequencia.keys()),
                                   "Frequência": list(frequencia.values())})
    df_frequencia = df_frequencia.nlargest(columns = "Frequência", n = quantidade)
    plt.figure(figsize=(12,8))
    ax = sns.barplot(data = df_frequencia, x = "Palavra", y = "Frequência", color = 'gray')
    ax.set(ylabel = "Contagem")
    plt.show()

In [ ]:
def word_cloud(text, column_text):
      all_words = ' '.join([text for text in text[column_text]])

      word_cloud = WordCloud(width = 800, height = 400,
                              max_font_size = 110, collocations = False).generate(all_words)

      plt.figure(figsize = (10,7))
      plt.imshow(word_cloud, interpolation = 'bilinear')
      plt.axis('off')
      plt.title('Nuvem de palavras:')
      plt.show()

In [ ]:
word_cloud(data, 'text')

In [ ]:
pareto(data,'text',10)

In [ ]:
def clean_tweets(tweet):
  tweet = BeautifulSoup(tweet, 'lxml').get_text()
  tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
  tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
  tweet = re.sub(r"[^a-zA-Z.!?]", ' ', tweet)
  tweet = re.sub(r" +", ' ', tweet)
  return tweet

In [ ]:
data['tratamento1'] = data['text'].apply(clean_tweets)

In [ ]:
word_cloud(data, 'tratamento1')

In [ ]:
pareto(data, "tratamento1", 10)

In [ ]:
nlp = sp.load('pt_core_news_sm')

In [ ]:
stop_words = sp.lang.pt.STOP_WORDS

In [ ]:
string.punctuation

In [ ]:
def clean_tweets2(tweet):

    tweet = tweet.lower()
    document = nlp(tweet)

    words = []

    for token in document:
        if token.text not in stop_words and token.text not in string.punctuation:
            words.append(token.text)

    # Juntar as palavras em uma string final
    cleaned_tweet = ' '.join(words)

    return cleaned_tweet

In [ ]:
text = 'hoje eu testei meus dados!'
clean_tweets2(text)

In [ ]:
data['tratamento2'] = data['tratamento1'].apply(clean_tweets2)

In [ ]:
data.head()

In [ ]:
word_cloud(data, 'tratamento2')

In [ ]:
pareto(data, "tratamento2", 10)

In [ ]:
stemmer = nltk.RSLPStemmer()


frase_processada = list()
for opiniao in data['tratamento2']:
    nova_frase = list()
    palavras_texto = token_espaco.tokenize(opiniao)
    for palavra in palavras_texto:
        nova_frase.append(stemmer.stem(palavra))
    frase_processada.append(' '.join(nova_frase))
data['tratamento3'] = frase_processada

In [ ]:
data.head()

In [ ]:
word_cloud(data, 'tratamento3')

In [ ]:
pareto(data, "tratamento3", 10)

In [ ]:
data.drop(['text','tratamento1','tratamento2'], axis = 1, inplace = True)

In [ ]:
data.head()

In [ ]:
data_clean = data['tratamento3']

## Tokenização

In [ ]:
2**16

In [ ]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(data_clean, target_vocab_size = 2**16)

In [ ]:
tokenizer.vocab_size

In [ ]:
ids = tokenizer.encode('eu sou feliz')
print(ids)

In [ ]:
text = tokenizer.decode(ids)
print(text)

In [ ]:
data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]

In [ ]:
for _ in range(10):
  print(data_inputs[random.randint(0, len(data_inputs) -1 )])

## Padding

In [ ]:
max_len = max([len(sentence) for sentence in data_inputs])
max_len

In [ ]:
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value = 0,
                                                            padding = 'post',
                                                            maxlen=max_len)

In [ ]:
for _ in range(10):
  print(data_inputs[random.randint(0, len(data_inputs) - 1)])

### Divisão entre treino e teste

In [ ]:
train_inputs, test_inputs, train_labels, test_labels = train_test_split(data_inputs,
                                                                        data_labels,
                                                                        test_size = 0.3,
                                                                        stratify = data_labels)

In [ ]:
test_labels.to_csv('test_labels.csv', index = False)

In [ ]:
train_labels.to_csv('train_labels.csv', index = False)

In [ ]:
np.savetxt('train_inputs.csv', train_inputs)

In [ ]:
np.savetxt('test_inputs.csv', test_inputs)

In [ ]:
data.to_csv('data.csv', index = False)

In [ ]:
data.head()

In [ ]:
train_labels.shape

In [ ]:
test_labels.shape

In [ ]:
train_inputs.shape

In [ ]:
test_inputs.shape

In [ ]:
len(set(train_labels))

## Etapa 2: Pré-Processamento de dados - Teste

In [ ]:
test_data = pd.read_csv('BaseTweets.csv')
test_data.drop_duplicates(inplace = True)

In [ ]:
test_data.head()

In [ ]:
test_data.rename(columns = {'text':'resenha'}, inplace = True)
test_data.rename(columns = {'createdAt':'datapublicacao'}, inplace = True)
test_data.rename(columns = {'retweetatCount':'retweets'}, inplace = True)
test_data.rename(columns = {'viewCount':'views'}, inplace = True)

In [ ]:
test_data['views'].fillna(test_data['views'].mean(), inplace = True)
test_data['views'] = test_data['views'].astype(int)

In [ ]:
test_data

In [ ]:
  test_data['datapublicacao'] = pd.to_datetime(test_data['datapublicacao'], format = '%a %b %d %H:%M:%S %z %Y')
  test_data = test_data.sort_values('datapublicacao')
  test_data['datapublicacao'] = test_data['datapublicacao'].dt.strftime('%d/%m/%Y')

In [ ]:
test_data

In [ ]:
data_clean = [clean_tweets2(clean_tweets(tweet)) for tweet in X]

In [ ]:
for _ in range(10):
  print(data_clean[random.randint(0, len(data_clean) - 1)])